In [ ]:
from http.server import BaseHTTPRequestHandler
from http.server import HTTPServer
import json

def start(handler_class = BaseHTTPRequestHandler):
    server_adress = ('', 8000)
    httpd = HTTPServer(server_adress, handler_class)
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        httpd.server_close()
       
class MyHTTPHandler(BaseHTTPRequestHandler):
    tasks = []
    try:
        index = 0
        with open('tasks.txt', 'r') as f:
            for row in f:
                row = row[1:-2]
                row_in_dict = dict(subrow.split(': ') for subrow in row.split(', '))
                tasks.append(row_in_dict)
                print(tasks[index])
                index+=1
    except FileNotFoundError:
        print('File not found')
        
    def do_GET(self):
        if self.path.startswith("/tasks"):
            self.send_response(200)
            self.send_header("Content-type", "text/html")
            self.end_headers()
            self.wfile.write("<html><head><meta charset='utf-8'>".encode())
            self.wfile.write("<title>Super</title></head>".encode())
            self.wfile.write(f"<body>{json.dumps(self.tasks)}</body></html>".encode())
    def do_POST(self):
        if self.path.endswith("/complete"):
            id = self.path.split('/')
            if int(id[2]) < len(self.tasks):
                if "'IsDone'" in self.tasks[int(id[2])]:
                    self.tasks[int(id[2])]["'IsDone'"] = "True"
                    self.send_response(200)
                    rewrite_tasks(self.tasks)
                else:
                    self.tasks[int(id[2])]["IsDone"] = "True"
                    self.send_response(200)
                    rewrite_tasks(self.tasks)
            else:
                self.send_response(404)    
        if self.path.endswith("/tasks"):
            data_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(data_length)
            post_data = json.loads(post_data.decode('utf-8'))
            index = len(self.tasks)
            post_data["IsDone"] = "False"
            post_data["Id"] = str(index)
            self.tasks.append(post_data)
            rewrite_tasks(self.tasks)
            self.send_response(200)
            self.send_header("Content-type", "text/html")
            self.end_headers()
            self.wfile.write("<html><head><meta charset='utf-8'>".encode())
            self.wfile.write("<title>Super</title></head>".encode())
            self.wfile.write(f"<body>{json.dumps(post_data)}</body></html>".encode())
        
def rewrite_tasks(tasks):
    with open('tasks.txt', 'w'):
        pass
    with open('tasks.txt', 'w') as f:
        for element in tasks:
            f.write(str(element)+'\n')
            
            
start(MyHTTPHandler)    